In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['Fraud_Data.csv', 'IpAddress_to_Country.csv']


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('../input/Fraud_Data.csv')
ip_addresses = pd.read_csv('../input/IpAddress_to_Country.csv')

In [4]:
data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 11 columns):
user_id           151112 non-null int64
signup_time       151112 non-null object
purchase_time     151112 non-null object
purchase_value    151112 non-null int64
device_id         151112 non-null object
source            151112 non-null object
browser           151112 non-null object
sex               151112 non-null object
age               151112 non-null int64
ip_address        151112 non-null float64
class             151112 non-null int64
dtypes: float64(1), int64(4), object(6)
memory usage: 12.7+ MB


In [6]:
data['signup_time'] = pd.to_datetime(data['signup_time'])
data['purchase_time'] = pd.to_datetime(data['purchase_time'])
data['source'] = data['source'].astype('category')
data['browser'] = data['browser'].astype('category')
data['sex'] = data['sex'].astype('category')
data['class'] = data['class'].astype('category')

In [7]:
data['class'].value_counts()

0    136961
1     14151
Name: class, dtype: int64

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 11 columns):
user_id           151112 non-null int64
signup_time       151112 non-null datetime64[ns]
purchase_time     151112 non-null datetime64[ns]
purchase_value    151112 non-null int64
device_id         151112 non-null object
source            151112 non-null category
browser           151112 non-null category
sex               151112 non-null category
age               151112 non-null int64
ip_address        151112 non-null float64
class             151112 non-null category
dtypes: category(4), datetime64[ns](2), float64(1), int64(3), object(1)
memory usage: 8.6+ MB


In [9]:
data_country = []
for index in data.index:
    tmp = ip_addresses.loc[(ip_addresses.loc[:,'upper_bound_ip_address'] > data.loc[index,'ip_address']) & 
                           (ip_addresses.loc[:,'lower_bound_ip_address'] < data.loc[index,'ip_address'])
                          ,'country'].values
    if len(tmp) == 1:
        data_country.append(''.join(tmp))
    else:
        data_country.append(None)

In [10]:
data['country'] = data_country
data['country'] = data['country'].astype('category')

In [11]:
data['purchase_signup_diff'] =  (data['purchase_time'] - data['signup_time']).dt.seconds

In [12]:
tmp = data.iloc[:,[0,4]].groupby('device_id').transform(len)
data['device_id_count'] = tmp

In [13]:
data['device_id_count'] = 0
data['device_id_count'] = data.iloc[:,[0,4]].groupby('device_id').transform(len)

In [14]:
data['signup_time_wd'] = data['signup_time'].dt.dayofweek
data['purchase_time_wd'] = data['purchase_time'].dt.dayofweek

data['signup_time_wy'] = data['signup_time'].dt.week
data['purchase_time_wy'] = data['purchase_time'].dt.week

In [15]:
data_rf = data.drop(columns=data.columns[[0,1,2,4]])

In [16]:
data_rf['country'] = data_rf['country'].replace(to_replace='NA',value='Not_found')

In [17]:
data_rf['country'] = data_rf['country'].astype('category')

In [18]:
data_rf['country'].cat.add_categories(['Other'],inplace=True)
data_rf.loc[data_rf.loc[:,'country'].isin(data['country'].value_counts()[50:].index.values),['country']] = 'Other'
data_rf.country = data_rf.country.cat.remove_unused_categories()

In [19]:
data_rf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 14 columns):
purchase_value          151112 non-null int64
source                  151112 non-null category
browser                 151112 non-null category
sex                     151112 non-null category
age                     151112 non-null int64
ip_address              151112 non-null float64
class                   151112 non-null category
country                 129146 non-null category
purchase_signup_diff    151112 non-null int64
device_id_count         151112 non-null int64
signup_time_wd          151112 non-null int64
purchase_time_wd        151112 non-null int64
signup_time_wy          151112 non-null int64
purchase_time_wy        151112 non-null int64
dtypes: category(5), float64(1), int64(8)
memory usage: 11.1 MB


In [20]:
data_rf.head()

,purchase_value,source,browser,sex,age,ip_address,class,country,purchase_signup_diff,device_id_count,signup_time_wd,purchase_time_wd,signup_time_wy,purchase_time_wy
0,34,SEO,Chrome,M,39,7.327584e+08,0,Japan,13882,1,1,5,9,16
1,16,Ads,Chrome,F,53,3.503114e+08,0,United States,17944,1,6,0,23,24
2,15,SEO,Opera,M,53,2.621474e+09,1,United States,1,12,3,3,1,1
3,44,SEO,Safari,M,41,3.840542e+09,0,NaN,60085,1,1,0,18,19
4,39,Ads,Safari,M,45,4.155831e+08,0,United States,41461,1,1,2,30,37


In [21]:
X = pd.get_dummies(data.drop(columns='class'))
y = data['class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.66, shuffle=False)

/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
clf = RandomForestClassifier(n_estimators=50, 
                            max_features=3, 
                            warm_start=True,
                            n_jobs=-1,
                            oob_score=True)

clf.fit(X_train, y_train)

In [ ]:
print clf.oob_score_

y_pred = clf.predict(X_train)
cfm = confusion_matrix(y_train, y_pred)
plt.figure()
sns.heatmap(cfm, annot=True, fmt="d", linewidths=.5, center=cfm.mean());

print clf.score(X_test, y_test)

y_pred = clf.predict(X_test)
cfm = confusion_matrix(y_test, y_pred)
plt.figure()
sns.heatmap(cfm, annot=True, fmt="d", linewidths=.5, center=cfm.mean());

In [ ]:
rf_results = pd.DataFrame(data={'true_values' : y_test,
                                'predictions' : y_pred
                               }
                         )

In [ ]:
y_score = clf.decision_path(X_test)
fpr = dict()
tpr = dict()
roc_auc = dict()
    
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()